In [2]:
import numpy as np
import pandas as pd

In [6]:
df = pd.read_csv('./Source/feature_all.csv', index_col='pair_id')
df.drop(columns=['mra', 'mean'], inplace=True)
df.head()

,name_1,name_2,l,dl,ssk,ts,tsr,pr,wr,smr,...,tdc,tdc_1,tdc_2,bag,mes,mess,rws,pqgdc,pqgjc,is_duplicate
pair_id,,,,,,,,,,,,,,,,,,,,,
1,iko industries,enormous industrial trade,0.564103,0.381090,0.589480,0.56,0.56,0.71,0.64,0.564103,...,0.452911,0.806060,0.524142,0.480000,0.766667,0.719872,0.646835,0.04878,0.025,0
2,apcotex industries,technocraft industries india,0.608696,0.424665,0.671847,0.52,0.71,0.72,0.86,0.565217,...,0.513436,0.893188,0.591080,0.571429,0.789474,0.765426,0.620774,0.00000,0.000,0
3,rishichem distributors,dsa,0.160000,0.119396,0.052106,0.16,0.16,0.67,0.60,0.080000,...,0.000000,0.326599,0.000000,0.090909,0.195652,0.347826,0.079790,0.00000,0.000,0
4,powermax rubber factory,co one,0.206897,0.157061,0.108644,0.34,0.34,0.33,0.43,0.137931,...,0.000000,0.474342,0.000000,0.217391,0.333333,0.416667,0.098339,0.00000,0.000,0
5,tress,longyou industries park zhejiang,0.216216,0.143389,0.156880,0.22,0.22,0.80,0.72,0.216216,...,0.179605,0.425210,0.174078,0.156250,0.666667,0.446970,0.522295,0.00000,0.000,0


In [7]:
df.columns[2:]

Index(['l', 'dl', 'ssk', 'ts', 'tsr', 'pr', 'wr', 'smr', 'sd', 'jd', 'js',
       'jws', 'tdc', 'tdc_1', 'tdc_2', 'bag', 'mes', 'mess', 'rws', 'pqgdc',
       'pqgjc', 'is_duplicate'],
      dtype='object')

In [10]:
from sklearn.model_selection import train_test_split #, GridSearchCV
# from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb

In [14]:
x_train, x_test, y_train, y_test = train_test_split(df.iloc[:, 2:-1], df.iloc[:, -1], test_size=0.3, random_state=42)

In [15]:
model = xgb.XGBClassifier()
model.fit(x_train, y_train)
y_pred = model.predict_proba(x_test)[:, 1]

In [17]:
from sklearn.metrics import precision_recall_curve, confusion_matrix

In [24]:
precision, recall, thresholds = precision_recall_curve(y_test, y_pred)
b = 1
fscore = (1 + b**2) * precision * recall / (b**2 * precision + recall)
ix = np.argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))

Best Threshold=0.280529, F-Score=0.738, Precision=0.719, Recall=0.757


In [26]:
precision, recall, thresholds = precision_recall_curve(y_test, y_pred)
b = 0.2 # 0.7
fscore = (1 + b**2) * precision * recall / (b**2 * precision + recall)
ix = np.argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))

Best Threshold=0.791277, F-Score=0.886, Precision=0.938, Recall=0.374


In [27]:
input_str = 'Bridgestone De Costa Rica Sociedad Anoni'

In [35]:
import re
import unicodedata
from cleanco import basename
from transliterate.decorators import transliterate_function


def preprocessing(x: str) -> str:
    
    def rus_preprocess(x: str) -> str:
        forms_of_ownership = [
            'ооо',
            'оао',
            'общество с ограниченной ответственностью',
            'открытое акционерное общество',
            'филиал компании'
        ]
        
        for form in forms_of_ownership:
            pattern = re.compile(form)
            x = pattern.sub('', x)
            
        return x
    
    @transliterate_function(language_code='ru', reversed=True)
    def translit(x: str) -> str:
        return x
    
    
    x = x.strip().lower()
    x = basename(x) if not bool(set(rus_alphabet).intersection(set(x.lower()))) else rus_preprocess(x)
    x = unicodedata.normalize('NFKD', x).encode('ASCII', 'ignore').decode() if not bool(set(rus_alphabet).intersection(set(x.lower()))) else translit(x)
    x = basename(x)
    x = re.sub(r'[^\w\s]',' ', x)
    
    return ' '.join([s for s in x.split()]) if len(x.split()) != 0 else '-' * 5
    

a = ord('а') # кирилическая "А"
rus_alphabet = ''.join([chr(i) for i in range(a,a+6)] + [chr(a+33)] + [chr(i) for i in range(a+6,a+32)])
    
input_str = preprocessing(input_str)
test_df = df[['name_2', 'is_duplicate']].copy()
test_df.name_2 = test_df.name_2.apply(lambda x: preprocessing(x))

In [41]:
import difflib # встроенная библиотека

import abydos.distance as abd
from fuzzywuzzy import fuzz
from strsimpy.cosine import Cosine
import Levenshtein, distance, jellyfish, textdistance

import warnings

warnings.filterwarnings("ignore")


def tdc_n(seq1, seq2, n):
    cosine = Cosine(n)
    p1 = cosine.get_profile(seq1)
    p2 = cosine.get_profile(seq2)
    return cosine.similarity_profiles(p1, p2)


feature_df = test_df.copy()

In [42]:
# Levenshtein
feature_df.insert(1, 'l', test_df.name_2.apply(lambda x: Levenshtein.ratio(input_str, x)))

In [44]:
# Discounted_Levenshtein
feature_df.insert(2, 'dl', test_df.name_2.apply(lambda x: abd.DiscountedLevenshtein().sim(input_str, x)))

# String Subsequence Kernel Similarity
feature_df.insert(3, 'ssk', test_df.name_2.apply(lambda x: abd.SSK().sim(input_str, x)))

# Token Sort Ratio
feature_df.insert(4, 'ts', test_df.name_2.apply(lambda x: fuzz.token_sort_ratio(input_str, x) / 100))

# Token Set Ratio
feature_df.insert(5, 'tsr', test_df.name_2.apply(lambda x: fuzz.token_set_ratio(input_str, x) / 100))

# Partial Ratio
feature_df.insert(6, 'pr', test_df.name_2.apply(lambda x: fuzz.partial_ratio(input_str, x) / 100))

# Weighted Ratio
feature_df.insert(7, 'wr', test_df.name_2.apply(lambda x: fuzz.WRatio(input_str, x) / 100))

# Sequence Matcher Ratio
feature_df.insert(8, 'smr', test_df.name_2.apply(lambda x: difflib.SequenceMatcher(None, input_str, x).ratio()))

In [ ]:
# нужно сохранять промежуточный результат

In [45]:
# Sorensen Distance
feature_df.insert(10, 'sd', test_df.name_2.apply(lambda x: 1 - distance.sorensen(input_str, x)))

# Jaccard Distance
feature_df.insert(11, 'jd', test_df.name_2.apply(lambda x: 1 - distance.jaccard(input_str, x)))

# Jaro Similarity
feature_df.insert(12, 'js', test_df.name_2.apply(lambda x: jellyfish.jaro_similarity(input_str, x)))

# Jaro-Winkler Similarity
feature_df.insert(13, 'jws', test_df.name_2.apply(lambda x: jellyfish.jaro_winkler_similarity(input_str, x)))

# Match Rating Approach
feature_df.insert(14, 'mra', test_df.name_2.apply(lambda x: int(False if jellyfish.match_rating_comparison(input_str, x) is None else jellyfish.match_rating_comparison(input_str, x))))

# Text Distance Cosine
feature_df.insert(15, 'tdc', test_df.name_2.apply(lambda x: 1-textdistance.Cosine(qval=2).distance(input_str, x)))

# Text Distance Cosine n=1
feature_df.insert(16, 'tdc_1', test_df.name_2.apply(lambda x: tdc_n(input_str, x, 1)))

# Text Distance Cosine n=2
feature_df.insert(17, 'tdc_2', test_df.name_2.apply(lambda x: tdc_n(input_str, x, 2)))

TypeError: str argument expected

In [ ]:
# нужно сохранять промежуточный результат

In [ ]:
# Bag
feature_df.insert(18, 'bag', test_df.name_2.apply(lambda x: abd.Bag().sim(input_str, x, 2)))

# Monge-Elkan similarity
feature_df.insert(19, 'mes', test_df.name_2.apply(lambda x: abd.MongeElkan(symmetric=False).sim(input_str, x, 2)))

# Monge-Elkan symmetric similarity
feature_df.insert(20, 'mess', test_df.name_2.apply(lambda x: abd.MongeElkan(symmetric=True).sim(input_str, x, 2)))

# Rouge-W similarity
feature_df.insert(21, 'rws', test_df.name_2.apply(lambda x: abd.RougeW().sim(input_str, x, 2)))

# Positional Q-Gram Dice coefficient
feature_df.insert(22, 'pqgdc', test_df.name_2.apply(lambda x: abd.PositionalQGramDice().sim(input_str, x, 2)))

# Positional Q-Gram Jaccard coefficient
feature_df.insert(23, 'pqgjc', test_df.name_2.apply(lambda x: abd.PositionalQGramJaccard().sim(input_str, x, 2)))